## Configurações padrão para geração do Dataset
#### Métodos  disponíveis
- SMA => Média móvel simples = $\frac{1}{n} \sum_{i=0}^{n-1} Price_{t-x}$ 
    - x => tamanho da janela de amostragem
- EMA => Média móvel exponencial = $\alpha (currentPrice - EMA_{t-1}) + EMA_{t-1}$
    - $\alpha = \frac{2}{n+1}$ => Fator de suavização
    - n => número de amostras na análise
    - $EMA_{t-1}$ => média móvel exponencial anterior
- MACD => Convergência/Divergência da média móvel = $EMA(n) - EMA(k)$
    - n => média exponencial rápida
    - k => média exponencial lenta
- CCI => Índice de Canal de Commodities = $\frac{TP - SMA(TP,N)}{0.015 * DP(TP)}$
    - DP => Desvio padrão de TP
    - SMA => Média móvel simples
    - TP => Preço típico (uma média simples entre os valores de fechamento, alta e baixa)
- ADX => Indicador de força da tendência = $\frac{(n-1)*EMA(TR) + TR}{n}$
    - TR => True Range = $max(H-L, abs(H-C_{t-1}), abs(L-C_{t-1}))$
- MTM => Indicador de momento = $Price_{t}-Price_{t-n}$
- ROC => Taxa de variação = $\frac{Price_{t}-Price_{t-n}}{Price_{t-n}}100$
- TSI => Indicador de Força Real = $\frac{EMA(EMA(PC,n), k)}{EMA(EMA(|PC|,n), k)}100$
    - PC => Variação do preço de fechamento = $price_{t-1} - price_t$
- K (%K) => Oscilador estocástico = $\frac{C_t-L_{t-n}}{H_{t-n}-L_{t-n}}$
    - C => Preço de fechamento
    - L => Menor valor
    - H => Maior valor
- D (%D) => Média móvel simples do Oscilador estocástico = $\frac{\sum_{i=0}^{n-1} \%K_{t-i}}{n}$
    - %K => Oscilador estocástico
- R (%R) => Williams %R = $\frac{H_{t-n}-C_t}{H_{t-n}-L_{t-n}}$ 
    - C => Preço de fechamento
    - L => Menor valor
    - H => Maior valor

In [1]:
inputDataName = "PETR4_B_0_30min"
output  = 'Fechamento'

# Disponíveis = ['SMA','EMA','MACD','CCI', 'ADX','MTM','ROC','TSI','K','D','R']
methods = ['MTC', 'SMA','EMA','MACD','CCI', 'ADX','ROC','TSI','K','D','R']
# methods = ['R', 'D']

# MTC - Moving time considered (quantos dias do passado serão utlizados nos dados de entrada)
daysQtd = 0
if ('MTC' in methods):
    daysQtd = 7
m = 4*daysQtd
cols = [m, m+1, m+2, m+3];

# SMA - Simple Moving Average
colSMA     = cols                             # Colunas de 'data' que serão utilizados para o calculo
windowSMA  = [3, 5, 15, 30, 200]              # A operação será realizada para cada granularização em cada coluna

# EMA - Exponential Moving Average
colEMA     = cols                             # Colunas de 'data' que serão utilizados para o calculo
windowEMA  = [5, 7, 9, 12, 26]                # A operação será realizada para cada granularização em cada coluna

# MACD - Moving Average Convergence/Divergence
colMACD    = cols                             # Colunas de 'data' que serão utilizados para o calculo
meanFast   = [ 4,  8, 12]                     # Valor da granulização da média móvel rápida
meanSlow   = [22, 17, 26]                     # Valor da granulização da média móvel lenta

# CCI - Commodity Channel Index
colCCI     = [m+1, m+2, m+3];                 # Tem normalmente como entrada os valores de HLC
windowCCI  = [14, 17, 18, 20]                 # valor da granulização do modelo

# ADX - Average Directional Index
colADX     = [m+1, m+2, m+3];                 # Tem como entrada os valores de HL (nessa mesma ordem)
windowADX  = [7, 14]                          # Fator de amortecimento considerado para as EMA (normalmente 14)

# MTM - Momentum indicator
colMTM     = cols                             # Colunas de 'data' que serão utilizados para o calculo
windowMTM  = [14,13]                          # A operação será realizada para cada granularização em cada coluna

# ROC - Price Rate of Change
colROC     = cols                             # Colunas de 'data' que serão utilizados para o calculo
windowROC  = [5, 10, 11, 12]                  # A operação será realizada para cada granularização em cada coluna

# TSI - True Strength Index
colTSI        = cols                          # Colunas de 'data' que serão utilizados para o calculo
fastWindTSI   = [13]                          # Janela para média exponencial rápida
slowWindTSI   = [25]                          # Janela para média exponencial lenta

# %K - Stochastic Oscillator
colK          = [m+1, m+2, m+3];              # Inserir dados de Máximo, Mínimo e Fechamento (HLC)
windowK       = [8, 10, 14]                   # Espaçamento entre as amostras (normalmente 14)

# %D - Stochastic Oscilator Average
colD          = [m+1, m+2, m+3];              # Inserir dados de Máximo, Mínimo e Fechamento (HLC)
windowD       = [14]                          # Espaçamento entre as amostras (normalmente 14) para o %K
windowDP      = [3]                           # Espaçamento entre as amostras para média simples

# %R - Williams
colR          = [m+1, m+2, m+3];              # Inserir dados de Máximo, Mínimo e Fechamento (HLC)
windowR       = [5, 14, 21]                   # Espaçamento entre as amostras (normalmente 14)


## Pacotes utilizados

In [2]:
import pandas as pd

## Funções de transformação dos dados

In [3]:
# ************************************* MTC - Moving time considered **************************************
def mtc(datas, days):
    rowq, colq = datas.shape
    
    colsName = [f'{datas.iloc[:,c].name}|T-{days}|' for c in range(colq)]
    res = pd.DataFrame(datas.values, columns = colsName)
    
    for q in range(days-1,-1,-1):
        colsName = [f'{datas.iloc[:,c].name}|T-{q}|' if q>0 else f'{datas.iloc[:,c].name}'
                    for c in range(colq)]
        newData = pd.DataFrame(datas.iloc[days-q:,:].values, columns = colsName)
        
        res = pd.concat([res, newData], axis=1)
    return res

# ************************************** SMA - Simple Moving Average **************************************
def sma(datas, windows):
    rowsq, colsq = datas.shape
    res = pd.DataFrame({})
    for c in range(colsq):
        colsName = [f'{datas.iloc[:,c].name}_SMA_{w}' for w in windows]
        newCols  = pd.DataFrame({}, columns = colsName) 
        cont = 0
        for w in windows:
            newCols[colsName[cont]] = datas.iloc[:,c].rolling(window=w).mean()
            cont += 1
        
        res = pd.concat([res, newCols], axis=1)  
    return res

# ************************************ EMA - Exponetial Moving Average ***********************************
def ema(datas, windows):
    rowsq, colsq = datas.shape
    res = pd.DataFrame({})
    for c in range(colsq):
        colsName = [f'{datas.iloc[:,c].name}_EMA_{w}' for w in windows]
        newCols  = pd.DataFrame({}, columns = colsName) 
        cont = 0
        for w in windows:
            newCols[colsName[cont]] = datas.iloc[:,c].ewm(span=w).mean()
            cont += 1
        
        res = pd.concat([res, newCols], axis=1)  
    return res

# ******************************* MACD - Moving Average Convergence/Divergence ***************************
def macd(datas, fast, slow):
    rowsq, colsq = datas.shape
    res = pd.DataFrame({})
    for c in range(colsq):
        colsName = [f'{datas.iloc[:,c].name}_MACD_{fast[i]}:{slow[i]}' for i in range(len(fast))]
        newCols  = pd.DataFrame({}, columns = colsName) 
        
        for i in range(len(fast)):
            newCols[colsName[i]] = (datas.iloc[:,c].ewm(span=fast[i]).mean() 
                                    - datas.iloc[:,c].ewm(span=slow[i]).mean())
        
        res = pd.concat([res, newCols], axis=1)  
    return res

# ************************************ CCI - Commodity Channel Index **************************************
def cci(datas, window):
    TP = datas.mean(axis = 1)
    display(TP)
    res = pd.DataFrame({})
    for w in window:
        MA = TP.rolling(window=w).mean()
        DP = TP.rolling(window=w).std()
        newCols  = pd.DataFrame({f'HLC_CCI_{w}': (TP-MA)/(0.015*DP)}) 
        res = pd.concat([res, newCols], axis=1)  
    return res


# ************************************ ADX - Average Directional Index ***********************************
def adx(datas, window):    
    TR   = pd.Series([max(datas.iloc[i,0]-datas.iloc[i,1], abs(datas.iloc[i,0]-datas.iloc[i-1,2]), 
                          abs(datas.iloc[i,1]-datas.iloc[i-1,2])) for i in range(len(datas.iloc[:,0]))])
    TR[0] = TR[1]
    res = pd.DataFrame({})
    for w in window:
        ATR   = TR.ewm(span=w).mean()
        items = ['Nan'] 
        for i in range(len(ATR)-1):
            items.append((ATR[i]*(w-1)+TR[i+1])/(w))     
        newCols = pd.DataFrame({f'ADX_{w}':items})
        res = pd.concat([res, newCols], axis=1) 
    return res

# ************************************** MTM - Momentum indicator ****************************************
def mtm(datas, window):
    rowq, colq = datas.shape  
    res = pd.DataFrame({})
    for c in range(colq):
        for w in window:
            items = []
            for r in range(rowq):
                items.append(datas.iloc[r,c] - datas.iloc[r-w,c] if r-w > 0 else 'NaN') 
            newCols = pd.DataFrame({f'{datas.iloc[:,c].name}_MTM_{w}':items})
            res = pd.concat([res, newCols], axis=1) 
    return res

# ************************************ ROC - Price Rate of Change ****************************************
def roc(datas, window):
    rowq, colq = datas.shape  
    res = pd.DataFrame({})
    for c in range(colq):
        for w in window:
            items = []
            for r in range(rowq):
                items.append(100*((datas.iloc[r,c]-datas.iloc[r-w,c])/datas.iloc[r-w,c]) if r-w > 0 else 'NaN') 
            newCols = pd.DataFrame({f'{datas.iloc[:,c].name}_ROC_{w}':items})
            res = pd.concat([res, newCols], axis=1) 
    return res

# ************************************ TSI - True Strength Index *****************************************
def tsi(datas, fast, slow):
    rowq, colq = datas.shape
    res = pd.DataFrame({})
    for c in range(colq):
        PC  = [datas.iloc[l-1,c]-datas.iloc[l,c] if l>=1 else 'NaN' for l in range(rowq)]
        PCM = [abs(PC[i]) if i>=1 else 'NaN' for i in range(rowq)]
        for w in  range(len(fast)):
            num = (pd.Series(PC).ewm(span=fast[w]).mean()).ewm(span=slow[w]).mean()
            den = (pd.Series(PCM).ewm(span=fast[w]).mean()).ewm(span=slow[w]).mean()
            newCols = pd.DataFrame({f'{datas.iloc[:,c].name}_TSI_{fast[w]}:{slow[w]}':((num/den)*100)})  
            res = pd.concat([res, newCols], axis=1)
    return res

# ************************************** %K - Stochastic Oscillator ***************************************
def k(datas, window):
    rowq = len(datas.iloc[:,0])
    res = pd.DataFrame({})

    for w in window:
        items = [] 
        for i in range(rowq):
            denominador = datas.iloc[i-w,0]-datas.iloc[i-w,1]
            
            if denominador < 0.01:
                denominador = 0.01
                
            if i-w >= 0 and i-w < rowq:
                items.append((datas.iloc[i,2]-datas.iloc[i-w,1])/denominador)
            else:
                items.append('NaN')  # Valor padrão caso ocorra divisão por zero
            
        newCols = pd.DataFrame({f'K%_{w}': items})
        res = pd.concat([res, newCols], axis=1)
    
    return res

          
# *********************************** %D - Stochastic Oscilator Average ***********************************
def d(datas, windowK, windowMean):
    datas = k(datas, windowK)
    datas = sma(datas,windowMean)
    return datas

# ******************************************** %R - Williams **********************************************
def r(datas, window):
    rowq = len(datas.iloc[:, 0])
    res = pd.DataFrame({})
      
    for w in window:
        items = [] 
        for i in range(rowq):
            denominador = datas.iloc[i - w, 0] - datas.iloc[i - w, 1]
            if denominador < 0.01:
                denominador = 0.01  # Define o denominador como 0.01 se for igual a zero
            
            numerator = datas.iloc[i - w, 0] - datas.iloc[i, 2]
            ratio = numerator / denominador if i - w >= 0 and i - w < rowq else 'NaN'
            items.append(ratio)
                
        newCols = pd.DataFrame({f'R%_{w}': items})
        res = pd.concat([newCols, res], axis=1)
    
    return res



# ------------------------------------------- Generate -------------------------------------------------------
def generate(methods):
    datas  = []
    bounds = []
    if('SMA' in methods):
        datas.append(sma(data.iloc[:,colSMA], windowSMA))
        bounds.append(max(windowSMA))
    if('EMA' in methods):
        datas.append(ema(data.iloc[:,colEMA], windowEMA))
        bounds.append(max(windowEMA))
    if('MACD' in methods):
        datas.append(macd(data.iloc[:,colMACD], meanFast, meanSlow))
        bounds.append(max(max(meanFast), max(meanSlow)))
    if('CCI' in methods):
        datas.append(cci(data.iloc[:,colCCI], windowCCI))
        bounds.append(max(windowCCI))
    if('ADX' in methods):
        datas.append(adx(data.iloc[:,colADX], windowADX))
        bounds.append(max(windowADX))
    if('MTM' in methods):
        datas.append(mtm(data.iloc[:,colMTM], windowMTM))
        bounds.append(max(windowMTM))
    if('ROC' in methods):
        datas.append(roc(data.iloc[:,colROC], windowROC))
        bounds.append(max(windowROC))
    if('TSI' in methods):
        datas.append(tsi(data.iloc[:,colTSI], fastWindTSI, slowWindTSI))
        bounds.append(max(max(fastWindTSI), max(slowWindTSI)))
    if('K' in methods):
        datas.append(k(data.iloc[:,colK], windowK))
        bounds.append(max(windowK))
    if('D' in methods):
        datas.append(d(data.iloc[:,colD], windowD, windowDP))
        bounds.append(max(max(windowD), max(windowDP)))
    if('R' in methods):
        datas.append(r(data.iloc[:,colR], windowR))
        bounds.append(max(windowR))
    return {'datas':datas, 'bounds':bounds}
        
def saveCSV(datas, bounds, dataOut):
    data = pd.concat(datas, axis=1)
    data = data.iloc[(max(bounds)+1): ,:]
    dataOut = dataOut.iloc[(max(bounds)+1):,:]
    data.to_csv(f'Generated/In_{inputDataName}.csv', index=False, sep = ';')  
    dataOut.to_csv(f'Generated/Out_{inputDataName}.csv', index=False, sep = ';') 
    
    pd.DataFrame(data.loc[:,output]).to_csv(f'Selected/In_estatistico_{inputDataName}.csv', index=False, sep = ';')  


## Importando dados externos

In [4]:
data = pd.read_csv(f'Collected/{inputDataName}.csv', sep=";", encoding='latin1') 

# Realiza a filtragem na base de dados
data = data.iloc[:,3:7]
display(data.head(3))

# Convert a base de dados de string para float
def converter_para_float(valor):
    value = valor.replace(".", "")
    return float(value.replace(",", "."))
data = data.applymap(converter_para_float)

# Define qual é a saída desejada
rowq, colq = data.shape

dataOut = pd.DataFrame({'OutPut |T+1|' : (data.loc[1:,output])})
data = data.iloc[:rowq-1, :]

# Verifica se tem espaçamento de dados passados
if('MTC' in methods):
    data = mtc(data, daysQtd)
    dataOut = pd.DataFrame({'OutPut |T+1|' : (data.loc[1:rowq-daysQtd-2,output])})
    data = data.iloc[:rowq-daysQtd-2, :]
    
display(data.shape)
display(data.head(3))

,Abertura,Máximo,Mínimo,Fechamento
0,"11,45","11,54","11,43","11,54"
1,"11,57","11,63","11,43","11,45"
2,"11,81","11,81","11,71","11,76"


(7539, 32)

,Abertura|T-7|,Máximo|T-7|,Mínimo|T-7|,Fechamento|T-7|,Abertura|T-6|,Máximo|T-6|,Mínimo|T-6|,Fechamento|T-6|,Abertura|T-5|,Máximo|T-5|,...,Mínimo|T-2|,Fechamento|T-2|,Abertura|T-1|,Máximo|T-1|,Mínimo|T-1|,Fechamento|T-1|,Abertura,Máximo,Mínimo,Fechamento
0,11.45,11.54,11.43,11.54,11.57,11.63,11.43,11.45,11.81,11.81,...,11.71,11.72,11.70,11.77,11.70,11.74,11.64,11.72,11.60,11.70
1,11.57,11.63,11.43,11.45,11.81,11.81,11.71,11.76,11.77,11.82,...,11.70,11.74,11.64,11.72,11.60,11.70,11.70,11.71,11.64,11.64
2,11.81,11.81,11.71,11.76,11.77,11.82,11.75,11.81,11.72,11.78,...,11.60,11.70,11.70,11.71,11.64,11.64,11.61,11.70,11.61,11.70


### Chamada de funções

In [5]:
result = generate(methods)
result['datas'].append(data)
result['bounds'].append(0)

# res = pd.DataFrame({})
# for dt in result['datas']:
#     res = pd.concat([res, dt], axis=1)
# display(res.head(3))
# print("resShap: ", res.shape)

classve = [0 if dataOut.values[i][0] < dataOut.values[i-1][0] else 1 for i in range(1, dataOut.shape[0])]
classve.insert(0,0)
classfi = pd.DataFrame({'OutPut_class |T+1|' : classve})

dataOut.reset_index(drop=True, inplace=True)

print("Classfi: ", classfi.shape)
print("OutData: ", dataOut.shape)

dataOut = pd.concat([dataOut, classfi], axis=1)
display(dataOut.head(10))
    
saveCSV(result['datas'], result['bounds'], dataOut)

0       11.673333
1       11.663333
2       11.670000
3       11.623333
4       11.603333
          ...    
7534    11.613333
7535    11.603333
7536    11.633333
7537    11.636667
7538    11.596667
Length: 7539, dtype: float64

Classfi:  (7539, 1)
OutData:  (7539, 1)


,OutPut |T+1|,OutPut_class |T+1|
0,11.64,0
1,11.70,1
2,11.61,0
3,11.61,1
4,11.61,1
5,11.62,1
6,11.68,1
7,11.74,1
8,11.74,1
9,11.71,0


####  Como chamar cada função de forma individual
* sma(data.iloc[:,colSMA], windowSMA)
* ema(data.iloc[:,colEMA], windowEMA)
* macd(data.iloc[:,colMACD], meanFast, meanSlow)
* cci(data.iloc[:,colCCI], windowCCI)
* adx(data.iloc[:,colADX], windowADX)
* mtm(data.iloc[:,colMTM], windowMTM)
* roc(data.iloc[:,colROC], windowROC)
* tsi(data.iloc[:,colTSI], fastWindTSI, slowWindTSI)
* k(data.iloc[:,colK], windowK)
* d(data.iloc[:,colD], windowD, windowDP)
* r(data.iloc[:,colR], windowR)